<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/Mistral/Supervised_fine_tuning_(SFT)_of_an_LLM_using_Hugging_Face_tooling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Supervised fine-tuning (SFT) of an LLM

집에서 ChatGPT를 만들기 위해서는 세 가지 단계를 거쳐야 합니다:

1. 사전 훈련(Pre-training): 인터넷 규모의 데이터에서 다음 토큰을 예측하도록 대형 언어 모델(LLM)을 사전 훈련하는 단계입니다. 이 과정은 수천 개의 GPU 클러스터에서 수행되며, 그 결과물을 "기본 모델"이라고 부릅니다.
2. 지도학습 미세조정(SFT): 기본 모델을 유용한 어시스턴트로 전환하는 단계입니다.
3. 인간 선호도 기반 미세조정: 어시스턴트의 친근함, 유용성 및 안전성을 향상시키는 단계입니다.

이 노트북에서는 두 번째 단계인 지도학습 미세조정(SFT), 또는 지침 조정에 대해 설명할 것입니다.

지도학습 미세조정은 첫 번째 단계에서 얻은 "기본 모델"을 바탕으로, 인터넷 텍스트에서 다음 토큰을 예측하도록 사전 훈련된 모델을 "챗봇" 또는 "어시스턴트"로 전환하는 과정입니다. 이는 교차 엔트로피 손실을 사용하여 인간의 지침 데이터를 기반으로 모델을 미세조정하는 것을 의미합니다. 다시 말해, 모델은 여전히 다음 토큰을 예측하도록 훈련되지만, 이제는 "런던에서 할 수 있는 10가지 일은 무엇인가요?", "팬케이크를 만드는 방법은?", "코끼리에 대한 시를 작성해 주세요"와 같은 지침에 따라 유용한 답변을 생성하도록 유도됩니다.


이를 위해서는 인간 주석자가 유용한 완성 문장을 수집하고, 이를 바탕으로 모델을 훈련시켜야 합니다. 예를 들어, OpenAI는
[사람들을 고용하여, 해당 작업을 수행했습니다.](https://gizmodo.com/chatgpt-openai-ai-contractors-15-dollars-per-hour-1850415474) 이들은 "런던에서는 빅벤을 방문할 수 있습니다(...)"와 같은 지침에 따라 유용한 완성 문장을 생성하도록 요청받았습니다. 공개적으로 이용 가능한 SFT 데이터셋의 훌륭한 모음은 [여기](https://huggingface.co/collections/HuggingFaceH4/awesome-sft-datasets-65788b571bf8e371c4e4241a)에서 확인할 수 있습니다.

Notes:
* 전체 노트북은 Hugging Face에서 개발한 [Alignment Handbook](https://github.com/huggingface/alignment-handbook)의 주석이 달린 버전으로 볼 수 있으며
특히 Zephyr-7b-beta를 훈련하는 데 사용된 [recipe](https://github.com/huggingface/alignment-handbook/blob/main/recipes/zephyr-7b-beta/sft/config_lora.yaml)을 기반으로 합니다. 

* 이 노트북은 Transformers 라이브러리에서 사용할 수 있는 모든 디코더 전용 LLM에 적용됩니다. 이 노트북에서는 현재 작성 시점에서 가장 우수한 오픈 소스 대형 언어 모델 중 하나인 [Mistral-7B base model](https://huggingface.co/mistralai/Mistral-7B-v0.1)을 미세조정할 것입니다.


## Required hardware

이 노트북은 [Ampere architecture](https://en.wikipedia.org/wiki/Ampere_(microarchitecture))이상을 지원하며, 최소 24GB의 RAM을 탑재한 NVIDIA GPU에서 실행되도록 설계되었습니다. 

포함되는 GPU는 다음과 같습니다:

* NVIDIA RTX 3090, 4090
* NVIDIA A100, H100, H200

등등. 개인적으로 저는 24GB RAM을 탑재한 RTX 4090에서 이 노트북을 실행하고 있습니다.

암페어 아키텍처를 요구하는 이유는 [bfloat16 (bf16) format](https://en.wikipedia.org/wiki/Bfloat16_floating-point_format)을 사용하기 때문인데, 이는 Turing과 같은 이전 아키텍처에서는 지원되지 않습니다.

하지만 약간의 수정으로 float16 (fp16) 포맷을 사용하여 모델을 훈련할 수 있으며, 이는 이전 세대 GPU에서도 지원됩니다. 

예를 들어:

* NVIDIA RTX 2080
* NVIDIA Tesla T4
* NVIDIA V100.

bf16을 fp16으로 변경해야 하는 위치에 대한 주석이 추가되어 있습니다.

## Set-up environment

지도 학습 미세조정을 수행하기 위해 필요한 모든 🤗 도구들을 설치하는 것부터 시작하겠습니다. 우리는 다음을 사용할 것입니다:

* Transformers: 미세조정할 대형 언어 모델(LLM)을 위해
* Datasets: 🤗 허브에서 SFT 데이터셋을 로드하고 모델에 맞게 준비하기 위해
* BitsandBytes 및 PEFT: 소비자 하드웨어에서 모델을 미세조정하기 위해 [Q-LoRa](https://huggingface.co/blog/4bit-transformers-bitsandbytes)를 활용하는데, 이는 미세조정에 필요한 계산 자원을 크게 줄여주는 기술입니다.
* TRL: LLM 미세조정을 위한 유용한 Trainer 클래스를 포함하는 [library](https://huggingface.co/docs/trl/index)입니다.


이제 필요한 패키지들을 설치해 보겠습니다.

In [1]:
!pip install -q transformers[torch] datasets

In [2]:
!pip install -q bitsandbytes trl peft

우리는 또한 모델의 어텐션 계산을 가속화하는 [Flash Attention](https://github.com/Dao-AILab/flash-attention)을 설치합니다.


* --no-build-isolation 을 쓴 이유는 사람마다 CUDA , Pytorch 버전이 다르기 때문에,, 함부로쓰면 dependency 문제가 생길수도 있습니다~

In [1]:
!pip install flash-attn --no-build-isolation



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 5.3 MB/s eta 0:00:0000:01:00:01
  Preparing metadata (setup.py) ... -^C
anceled

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
ERROR: Operation cancelled by user


NameError: name 'no' is not defined

## Load dataset

Note :가이드는 여러 데이터셋을 혼합하여 각 데이터셋이 일정 비율의 학습 예제를 포함할 수 있도록 지원합니다. 하지만 Zephyr 레시피는 단일 데이터셋만 사용하며, 해당 데이터셋은 [UltraChat200k dataset](https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k)입니다.


In [4]:
from datasets import load_dataset

# based on config
raw_datasets = load_dataset("HuggingFaceH4/ultrachat_200k")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


데이터셋은 여러 분할로 나뉘어 있으며, 각 분할마다 특정 수의 행이 포함되어 있습니다. 저희는 감독된 미세 조정(SFT)을 진행할 예정이기 때문에 "train_sft"와 "test_sft" 분할만 사용합니다.

In [5]:
from datasets import DatasetDict

# remove this when done debugging
indices = range(0,100)

dataset_dict = {"train": raw_datasets["train_sft"].select(indices),
                "test": raw_datasets["test_sft"].select(indices)}

raw_datasets = DatasetDict(dataset_dict)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 100
    })
    test: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 100
    })
})

좋습니다. 하나의 예제를 확인해보겠습니다. 각 예제가 메시지 목록을 포함해야 한다는 점이 중요합니다.


In [6]:
example = raw_datasets["train"][0]
print(example.keys())

dict_keys(['prompt', 'prompt_id', 'messages'])


각 메시지는 두 개의 키를 포함하는 사전(dictionary)입니다:

* "role": 메시지의 작성자를 지정합니다. 값은 "system", "assistant", 또는 "user"일 수 있습니다. 여기서 "user"는 사람을 의미합니다.
* "content": 메시지의 실제 내용을 담고 있습니다.
다음은 이 학습 예제의 메시지 시퀀스를 출력한 예입니다:

In [7]:
messages = example["messages"]
for message in messages:
  role = message["role"]
  content = message["content"]
  print('{0:20}:  {1}'.format(role, content))

user                :  These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?
On your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!
Your Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.
Does this feature apply to all sections of the theme or just specific ones as listed in the text material?
assistant           :  This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.
user                :  Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?
assistant           :  Sure, here are the steps to enable the secondary 

이번 경우, 지침이 Shopify에서 특정 기능을 활성화하는 것에 관한 것 같습니다. 흥미롭군요!


(Shopify는 종합 상거래 플랫폼이다.)

## Load tokenizer

Next, we instantiate the tokenizer, which is required to prepare the text for the model. The model doesn't directly take strings as input, but rather `input_ids`, which represent integer indices in the vocabulary of a Transformer model. Refer to my [YouTube video](https://www.youtube.com/watch?v=IGu7ivuy1Ag&ab_channel=NielsRogge) if you want to know more about it.


다음으로, 모델을 위한 텍스트를 준비하는 데 필요한 토크나이저를 인스턴스화합니다. 모델은 문자열을 직접 입력으로 받지 않고, 대신 Transformer 모델의 어휘 사전에 있는 정수 인덱스를 나타내는 input_ids를 입력으로 받습니다. 이에 대해 더 알고 싶다면 [YouTube video](https://www.youtube.com/watch?v=IGu7ivuy1Ag&ab_channel=NielsRogge)를 참고하세요.


또한, 기본 모델의 토크나이저에는 일반적으로 설정되어 있지 않은 몇 가지 속성을 설정합니다. 예를 들어 :

- 패딩 토큰 ID: 사전 학습 동안에는 다음 토큰을 예측하기 위해 텍스트 블록을 생성하기 때문에 패딩이 필요 없지만, 미세 조정(fine-tuning) 시에는 (지시문, 완성) 쌍을 동일한 길이의 배치로 만들기 위해 패딩이 필요합니다.
- 모델 최대 길이: 모델에 너무 긴 시퀀스를 잘라내기 위해 필요합니다. 여기서는 최대 2048 토큰까지 학습하기로 결정했습니다.
- 채팅 템플릿: [chat template](https://huggingface.co/blog/chat-templates)은 각 메시지 목록이 <|user|>와 같은 특수 문자열을 추가하여 토큰화 가능한 문자열로 변환되는 방식을 결정합니다. 이는 사용자 메시지와 챗봇의 응답을 구분하는 역할을 합니다. 여기서는 대부분의 채팅 모델에서 사용하는 기본 채팅 템플릿을 정의합니다. 자세한 내용은 [docs](https://huggingface.co/docs/transformers/main/en/chat_templating)를 참조하세요.

In [8]:
from transformers import AutoTokenizer

model_id = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
    tokenizer.model_max_length = 2048

# Set chat template
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


## Apply chat template

토크나이저에 적절한 속성을 설정한 후에는 각 메시지 목록에 채팅 템플릿을 적용할 차례입니다. 여기서는 기본적으로 각 (지시문, 완성) 메시지 목록을 모델이 토큰화할 수 있는 문자열로 변환합니다.

`tokenize=False`로 설정한 점에 유의하세요. 이는 나중에 정의할 `SFTTrainer`가 내부적으로 토큰화를 수행할 것이기 때문입니다. 여기서는 메시지 목록을 동일한 형식의 문자열로 변환하는 작업만 수행합니다.

In [9]:
import re
import random
from multiprocessing import cpu_count

def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return example

column_names = list(raw_datasets["train"].features)
raw_datasets = raw_datasets.map(apply_chat_template,
                                num_proc=cpu_count(),
                                fn_kwargs={"tokenizer": tokenizer},
                                remove_columns=column_names,
                                desc="Applying chat template",)

# create the splits
train_dataset = raw_datasets["train"]
eval_dataset = raw_datasets["test"]

for index in random.sample(range(len(raw_datasets["train"])), 3):
  print(f"Sample {index} of the processed training set:\n\n{raw_datasets['train'][index]['text']}")

Sample 87 of the processed training set:


</s>

Write a news article covering the latest advancements in space exploration. Your article should include recent discoveries, technological developments, and scientific breakthroughs. Use a formal and objective writing style and interview experts in the field for additional insights. The article should also discuss the potential impact of these advancements on future space missions and exploration.</s>

In recent years, space exploration has reached new heights with exciting discoveries and technological advancements that have changed our understanding of the universe. From the discovery of exoplanets to the development of more efficient spacecraft, the latest breakthroughs in space exploration are guiding us towards a new era of space science and exploration.

One recent discovery that has rocked the space science world is the discovery of the TRAPPIST-1 system, which contains seven Earth-sized planets, three of which are within the habit

We also specified `remove_columns` to the map function above, meaning that we are now left with only 1 column: "text".

Hence the set-up is now very similar to pre-training: we will just train the model predict the next token, given the previous ones. In this case, the model will learn to generate completions given instructions.

Hence, similar to pre-training, the labels will be created automatically based on the inputs (by shifting them one position to the right). The model is still trained using cross-entropy. This means that evaluation will mostly be done by checking perplexity/validation loss/model generations.

In [10]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text'],
        num_rows: 100
    })
})

## Define model arguments

다음으로, 모델 인자를 정의할 시간입니다.

여기서는 모델을 미세 조정(fine-tuning)하는 다양한 방법에 대해 설명이 필요합니다.

### Full fine-tuning

일반적으로 "전체 미세 조정"을 수행합니다. 이는 미세 조정 중에 기본 모델의 모든 가중치를 업데이트한다는 의미입니다. 이는 일반적으로 전체 정밀도(float32) 또는 혼합 정밀도(float32와 float16의 조합)로 수행됩니다. 그러나 LLM과 같은 점점 더 큰 모델의 경우, 이는 실현 불가능해집니다.

참고로, float32는 모델의 각 매개변수가 32비트 또는 4바이트로 저장됨을 의미합니다. 예를 들어, Mistral-7B와 같은 70억 매개변수 모델의 경우, 70억 매개변수 × 매개변수당 4바이트 = 280GB의 GPU RAM이 필요합니다. AdamW와 같은 옵티마이저를 사용하여 훈련할 때는 모델뿐만 아니라 그래디언트와 옵티마이저 상태를 위한 메모리도 필요하며, 이는 혼합 정밀도로 훈련할 경우 모델 크기의 약 18배에 해당하는 기가바이트의 메모리가 필요합니다. 이 경우 7B × 18 = 126GB의 GPU RAM이 필요합니다. 이는 단지 70억 매개변수 모델의 경우에 불과합니다! 자세한 내용은 가이드를 참조하세요:https://huggingface.co/docs/transformers/v4.20.1/en/perf_train_gpu_one.

### LoRa, a PEFT method

따라서, Microsoft의 일부 영리한 연구진은 [LoRa](https://huggingface.co/docs/peft/conceptual_guides/lora) (low-rank adaptation)라는 방법을 고안해냈습니다. 여기서 아이디어는 전체 미세 조정을 수행하는 대신, 기존 모델을 고정하고 모델에 몇 개의 매개변수 가중치(“어댑터”라고 함)를 추가하여 이를 훈련시키는 것입니다.

oRa는 매개변수 효율적인 미세 조정(PEFT) 방법이라고 합니다. 이는 몇 개의 어댑터만 훈련시키고 기존 모델은 그대로 두어 매개변수 효율적인 방식으로 모델을 미세 조정하는 인기 있는 방법입니다. LoRa는 Hugging Face의  [PEFT library](https://huggingface.co/docs/peft/v0.7.1/en/index)에 포함되어 있으며, 다양한 다른 PEFT 방법도 지원합니다(하지만 작성 시점에서는 LoRa가 가장 인기 있는 방법입니다).

### QLoRa, an even more efficient method

일반적인 LoRa의 경우, 기본 모델을 메모리에 32비트 또는 16비트로 유지하고 매개변수 가중치를 훈련시킵니다. 그러나 매개변수당 8비트 또는 4비트로 모델 크기를 크게 줄이는 새로운 방법들이 개발되었습니다(이를  ["quantization"](https://huggingface.co/docs/transformers/main_classes/quantization)이라고 합니다). 따라서 LoRa를 양자화된 모델(예: 4비트 모델)에 적용하면 이를 QLoRa라고 부릅니다. 이에 대해 자세히 알고 싶다면 블로그 게시물을 참조하세요. 다양한 양자화 방법이 있지만, 여기서는 [BitsandBytes](https://huggingface.co/docs/transformers/main_classes/quantization#transformers.BitsAndBytesConfig) 통합을 사용할 것입니다.


In [11]:
from transformers import BitsAndBytesConfig
import torch

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
)
device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None

model_kwargs = dict(
    attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map=device_map,
    quantization_config=quantization_config,
)

## Define SFTTrainer



다음으로, TRL 라이브러리에서 제공하는 [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer)를 정의합니다. 이 클래스는 Transformers 라이브러리의 Trainer 클래스를 상속받지만, 감독된 미세 조정(Instruction Tuning)에 최적화되어 있습니다.[Accelerate](https://huggingface.co/docs/accelerate/index)를 백엔드로 사용하여 하나 이상의 GPU에서 즉시 훈련할 수 있습니다.

특히, SFTTrainer는 [packing](https://huggingface.co/docs/trl/sft_trainer#packing-dataset--constantlengthdataset-)을 지원합니다. 이는 여러 개의 짧은 예제를 동일한 입력 시퀀스에 패킹하여 훈련 효율성을 높이는 방법입니다.

우리가 QLoRa를 사용할 것이므로, PEFT 라이브러리는 어댑터를 적용할 기본 모델의 레이어를 정의하는 편리한 [LoraConfig](https://huggingface.co/docs/peft/v0.7.1/en/package_reference/lora#peft.LoraConfig)를 제공합니다. 일반적으로 Transformer의 어텐션 레이어에 있는 선형 프로젝션 매트릭스에 LoRa를 적용합니다. 그런 다음 이 구성을 SFTTrainer 클래스에 제공합니다. `model_id`를 지정함으로써 기본 모델의 가중치가 로드됩니다(이 과정은 시간이 소요됩니다).


또한, 다음과 같은 훈련 관련 다양한 하이퍼파라미터를 설정합니다:

* 에폭 동안 미세 조정: 전체 데이터셋을 한 번 학습합니다.
* 학습률 및 스케줄러: 학습률과 학습률 스케줄러를 설정하여 모델의 학습 과정을 조절합니다.
* 그래디언트 체크포인팅: 훈련 중 메모리를 절약하기 위해 그래디언트 체크포인팅을 사용합니다.
* 기타 설정: 배치 크기, 가중치 감소 등 추가적인 하이퍼파라미터를 설정합니다.


In [12]:
import torch
print(torch.cuda.is_available())  # True가 출력되어야 합니다
print(torch.cuda.device_count())  # 사용 가능한 GPU의 수를 출력합니다
print(torch.cuda.get_device_name(0))  # GPU 이름을 출력합니다

True
1
NVIDIA H100 80GB HBM3


In [13]:
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments

# 패딩 토큰 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# path where the Trainer will save its checkpoints and logs
output_dir = 'data/zephyr-7b-sft-lora'

# based on config
training_args = TrainingArguments(
    fp16=True, # specify bf16=True instead when training on GPUs that support bf16
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=128,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=10000,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=1, # originally set to 8
    per_device_train_batch_size=1, # originally set to 8
    # push_to_hub=True,
    # hub_model_id="zephyr-7b-sft-lora",
    # hub_strategy="every_save",
    # report_to="tensorboard",
    save_strategy="no",
    save_total_limit=None,
    seed=42,
)

# based on config
peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

trainer = SFTTrainer(
        model=model_id,
        model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        dataset_text_field="text",
        tokenizer=tokenizer,
        packing=True,  # 패킹 활성화
        peft_config=peft_config,
        max_seq_length=2048,  # max_length 값을 2048로 설정
    )

# 데이터셋 확인
print("Train Dataset:", len(train_dataset))
print("Eval Dataset:", len(eval_dataset))

# 훈련 시작
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': model_init_kwargs, dataset_text_field, packing, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be remove

Train Dataset: 100
Eval Dataset: 100


***** Running training *****
  Num examples = 65
  Num Epochs = 10,000
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 128
  Total optimization steps = 10,000
  Number of trainable parameters = 54,525,952
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.


Epoch,Training Loss,Validation Loss
1,No log,1.172302
2,No log,1.166906
3,No log,1.164446



***** Running Evaluation *****
  Num examples = 62
  Batch size = 1

***** Running Evaluation *****
  Num examples = 62
  Batch size = 1

***** Running Evaluation *****
  Num examples = 62
  Batch size = 1

***** Running Evaluation *****
  Num examples = 62
  Batch size = 1


## Train!

마지막으로, 훈련은 `trainer.train()`을 호출하는 것만큼 간단합니다!

In [ ]:
train_result = trainer.train()

## Saving the model

다음으로, Trainer의 상태를 저장합니다. 또한, 로그에 학습 샘플 수를 추가합니다.

In [ ]:
metrics = train_result.metrics
max_train_samples = training_args.max_train_samples if training_args.max_train_samples is not None else len(train_dataset)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

## Inference

훈련된 모델로 새로운 텍스트를 생성해보겠습니다.

추론에는 두 가지 주요 방법이 있습니다:
* [pipeline API](https://huggingface.co/docs/transformers/pipeline_tutorial)를 사용하면 전처리 및 후처리에 대한 많은 세부 사항을 추상화하여 간편하게 사용할 수 있습니다. 예를 들어, [model card](https://huggingface.co/HuggingFaceH4/mistral-7b-sft-beta#intended-uses--limitations)가 이를 잘 보여줍니다.
* `AutoTokenizer`와 `AutoModelForCausalLM` 클래스를 직접 사용하여 전처리 및 후처리 과정을 직접 구현합니다.

우리는 후자의 방법을 선택하여, 내부 동작 방식을 이해해보겠습니다.

먼저, 가중치를 저장한 디렉토리에서 모델을 로드합니다. 또한, 4비트 추론을 사용하고, 모델을 사용 가능한 GPU에 자동으로 배치하도록 지정합니다. (`device_map="auto"`에 대한 자세한 내용은 [documentation](https://huggingface.co/docs/accelerate/concept_guides/big_model_inference#the-devicemap)를 참고하세요).

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForCausalLM.from_pretrained(output_dir, load_in_4bit=True, device_map="auto")

다음으로, 토크나이저의 채팅 템플릿을 사용하여 모델에 전달할 메시지 목록을 준비합니다. 여기서 모델이 어떻게 행동해야 하는지를 지정하기 위해 "system" 메시지도 추가합니다. 훈련 중에는 모든 대화에 빈 시스템 메시지를 추가했습니다.

또한, `add_generation_prompt=True`를 지정하여 모델이 응답을 생성하도록 유도합니다(이는 추론 시에 유용합니다). 입력을 GPU로 이동시키기 위해 "cuda"를 지정합니다. 이전에 `device_map="auto"`를 사용했기 때문에 모델은 자동으로 GPU에 배치됩니다.

다음으로, [generate()](https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/text_generation#transformers.GenerationMixin.generate)를 사용하여 다음 토큰 ID를 하나씩 자기회귀적으로 생성합니다. 그리디 디코딩(greedy decoding)이나 빔 서치(beam search)와 같은 다양한 생성 전략이 있다는 점에 유의하세요. 자세한 내용은 [이 블로그](https://huggingface.co/blog/how-to-generate)를 참조하세요. 여기서는 샘플링(sampling)을 사용합니다.

마지막으로, 토크나이저의 batch_decode 메서드를 사용하여 생성된 토큰 ID를 다시 문자열로 변환합니다.

In [ ]:
import torch

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]

# prepare the messages for the model
input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

# inference
outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])